# Foodie Tour Workflow with Julep AI

This notebook creates a workflow that:
1. Checks today's weather for a list of cities
2. Suggests indoor or outdoor dining based on weather conditions
3. Picks 3 iconic local dishes per city
4. Finds top-rated restaurants serving these dishes
5. Creates a delightful one-day "foodie tour" narrative

## 1. Setup and Initialization

First, let's install the Julep client and set up our environment.

In [1]:
# Install Julep if needed
!pip install julep pyyaml -U --quiet 


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import required libraries
from julep import Client
import os
import uuid

# Generate unique IDs for our agent and task
AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4()

In [ ]:
# Initialize Julep client
# No need to create API key - Julep has a generous free tier
client = Client(api_key=os.environ['JULEP_API_KEY'])

# Create our agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name="Foodie Tour Guide",
    about="An AI assistant that creates personalized foodie tours based on weather conditions",
    model="gpt-4o",  # Using the latest model for best results
)

## 2. Define Tools and Integrations

Now, let's set up the tools our agent will use.

In [4]:
# Add weather integration tool
weather_tool = client.agents.tools.create(
    agent_id=agent.id,
    name='weather',
    type='integration',
    integration={'provider': 'weather'}
)

# Add internet search integration tool
search_tool = client.agents.tools.create(
    agent_id=agent.id,
    name='internet_search',
    type='integration',
    integration={'provider': 'brave'}
)

# Add Wikipedia integration tool
wiki_tool = client.agents.tools.create(
    agent_id=agent.id,
    name='wikipedia',
    type='integration',
    integration={'provider': 'wikipedia'}
)

## 3. Define Task Workflow

Let's create our task definition using YAML.

In [ ]:
import yaml

# Define API keys
OPENWEATHERMAP_API_KEY = "YOUR_OPENWEATHERMAP_API_KEY"
BRAVE_API_KEY = "YOUR_BRAVE_API_KEY"

# Define the task
task_def = yaml.safe_load(f"""
# yaml-language-server: $schema=https://raw.githubusercontent.com/julep-ai/julep/refs/heads/dev/schemas/create_task_request.json
name: Foodie Tour Creator
description: Creates a personalized foodie tour based on weather conditions for a list of cities.

########################################################
####################### INPUT SCHEMA ##################
########################################################
input_schema:
  type: object
  properties:
    cities:
      type: array
      items:
        type: string
      description: The list of cities to create foodie tours for.

########################################################
####################### TOOLS ##########################
########################################################

# Define the tools that the task will use in this workflow
tools:
- name: weather
  type: integration
  integration:
    provider: weather
    method: get
    setup:
      openweathermap_api_key: "{OPENWEATHERMAP_API_KEY}"

- name: internet_search
  type: integration
  integration:
    provider: brave
    setup:
      brave_api_key: "{BRAVE_API_KEY}"

- name: wikipedia
  type: integration
  integration:
    provider: wikipedia
    method: search

########################################################
####################### MAIN WORKFLOW ##########################
########################################################

main:
# Step 0: Fetch weather data for each city
- over: $ steps[0].input.cities
  map:
    tool: weather
    arguments:
      location: $ _

# Step 1: Search for iconic local dishes for each city
- over: $ steps[0].input.cities
  map:
    tool: internet_search
    arguments:
      query: $ 'iconic local dishes in ' + _

# Step 2: Search for top restaurants in each city
- over: $ steps[0].input.cities
  map:
    tool: internet_search
    arguments:
      query: $ 'best restaurants in ' + _

# Step 3: Search Wikipedia for culinary information about each city
- over: $ steps[0].input.cities
  map:
    tool: wikipedia
    arguments:
      query: $ _ + ' cuisine'
      load_max_docs: 2

# Step 4: Zip cities, weather, dishes, restaurants, and wiki info into a list of tuples
- evaluate:
    zipped: |-
      $ list(
        zip(
          steps[0].input.cities,
          [output['result'] for output in steps[0].output],
          steps[1].output,
          steps[2].output,
          steps[3].output
        )
      )

# Step 5: Create a foodie tour for each city
- over: $ _['zipped']
  parallelism: 3
  map:
    prompt:
    - role: system
      content: >-
        $ f'''You are {{agent.name}}, a culinary expert and travel guide. Your task is to create a delightful one-day "foodie tour" for a city.
        
        The user will give you the following information:
        - The city name
        - The current weather conditions
        - Information about iconic local dishes
        - Information about top restaurants
        - Wikipedia information about the city's cuisine
        
        Based on this information, you need to:
        1. Analyze the weather and suggest whether indoor or outdoor dining would be more appropriate
        2. Select exactly 3 iconic local dishes that represent the city's culinary heritage
        3. Recommend specific restaurants for each dish, focusing on authentic and highly-rated establishments
        4. Create a narrative for a one-day foodie tour with breakfast, lunch, and dinner recommendations
        5. Factor in the weather conditions when planning the tour
        
        Your response should be engaging, informative, and practical. Include specific restaurant names, dish descriptions, and suggestions for the best time to visit each place.'''
    - role: user
      content: >-
        $ f'''City: "{{_[0]}}"
        Weather: "{{_[1]}}"
        Local Dishes Information: "{{_[2]}}"
        Restaurant Information: "{{_[3]}}"
        Cuisine Background: "{{_[4]}}"'''
    unwrap: true

# Step 6: Create a final output by joining the foodie tours for each city
- evaluate:
    final_output: $ "\\n\\n====================\\n\\n".join(tour for tour in _)
""")

# Create the task
task = client.tasks.create_or_update(
    task_id=TASK_UUID,
    agent_id=agent.id,
    **task_def
)


## 4. Execute the Workflow

Now let's run our workflow with a list of cities.

In [32]:
# Define the list of cities
cities = ["New York", "Tokyo", "Paris", "Bangkok", "Rome"]
# cities = ['New York']

# Create an execution
execution = client.executions.create(
    task_id=task.id,
    input={"cities": cities}
)

print(f"Started execution with ID: {execution.id}")

Started execution with ID: 06841cfa-a0e0-7e7e-8000-0a21d419e754


## 5. Monitor Execution and Get Results

Let's monitor the execution and retrieve the results when it's complete.

In [33]:
import time

# Monitor execution status
execution = client.executions.get(execution.id)

while execution.status != "succeeded" and execution.status != "failed":
    time.sleep(5)
    execution = client.executions.get(execution.id)
    print(f"Execution status: {execution.status}")
    print("-" * 50)

# Get the final output
if execution.status == "succeeded":
    if 'final_output' in execution.output:
        print(execution.output['final_output'])
    else:
        print(execution.output)
else:
    print(f"Execution failed: {execution.error}")






# import pprint
# import time

# def reformat_output(output):
#     if isinstance(output, dict):
#         for key, value in output.items():
#             if (key == "base64_image" or key == "url") and value is not None:
#                 output[key] = value[:20] + '...'
#             else:
#                 output[key] = reformat_output(value)
#     elif isinstance(output, list):
#         for i, item in enumerate(output):
#             output[i] = reformat_output(item)
#     return output

# # Execute the task
# execution = client.executions.create(
#     task_id=task.id,
#     input={"cities": ["New York"]}  # Starting with just 2 cities for testing
# )

# print(f"Started execution with ID: {execution.id}")

# # Monitor execution with detailed transition information
# while execution.status != "succeeded" and execution.status != "failed":
#     time.sleep(5)
#     execution = client.executions.get(execution.id)
#     print(f"\nExecution status: {execution.status}")
#     print("-" * 50)
    
#     # Get detailed transition information
#     transitions = client.executions.transitions.list(execution_id=execution.id).items
    
#     for index, transition in enumerate(reversed(transitions)):
#         print(f"\nTransition Index: {index}")
#         print(f"Transition Type: {transition.type}")
#         print("Transition Output:")
#         pprint.pprint(reformat_output(transition.output))
#         print("-" * 100)

# # Final output
# if execution.status == "succeeded":
#     if 'final_output' in execution.output:
#         print("\nFinal Output:")
#         print(execution.output['final_output'])
#     else:
#         print("\nExecution Output:")
#         pprint.pprint(reformat_output(execution.output))
# else:
#     print(f"\nExecution failed: {execution.error}")


Execution status: running
--------------------------------------------------
Execution status: running
--------------------------------------------------
Execution status: running
--------------------------------------------------
Execution status: running
--------------------------------------------------
Execution status: running
--------------------------------------------------
Execution status: running
--------------------------------------------------
Execution status: running
--------------------------------------------------
Execution status: running
--------------------------------------------------
Execution status: running
--------------------------------------------------
Execution status: running
--------------------------------------------------
Execution status: running
--------------------------------------------------
Execution status: running
--------------------------------------------------
Execution status: succeeded
------------------------------------------------

## 6. Save Results to File (Optional)

Optionally, we can save the results to a file for future reference.

In [35]:
# Save results to file if execution succeeded
if execution.status == "succeeded" and 'final_output' in execution.output:
    with open('foodie_tour_results.md', 'w', encoding='utf-8') as f:
        f.write("# Foodie Tour Results\n\n")
        f.write(execution.output['final_output'])
    
    print("Results saved to foodie_tour_results.md")

Results saved to foodie_tour_results.md


In [37]:
# Lists all the task steps that have been executed up to this point in time
transitions = client.executions.transitions.list(execution_id=execution.id).items

# Transitions are retrieved in reverse chronological order
for transition in reversed(transitions):
    print("Transition type: ", transition.type)
    print("Transition output: ", transition.output)
    print("-"*50)

Transition type:  init
Transition output:  {'cities': ['New York', 'Tokyo', 'Paris', 'Bangkok', 'Rome']}
--------------------------------------------------
Transition type:  init_branch
Transition output:  New York
--------------------------------------------------
Transition type:  init_branch
Transition output:  Bangkok
--------------------------------------------------
Transition type:  init_branch
Transition output:  Paris
--------------------------------------------------
Transition type:  init_branch
Transition output:  Tokyo
--------------------------------------------------
Transition type:  init_branch
Transition output:  Rome
--------------------------------------------------
Transition type:  finish_branch
Transition output:  {'result': "In Paris, the current weather is as follows:\nDetailed status: moderate rain\nWind speed: 8.75 m/s, direction: 180°\nHumidity: 90%\nTemperature: \n  - Current: 15.96°C\n  - High: 16.77°C\n  - Low: 15.21°C\n  - Feels like: 15.96°C\nRain: {'1h